In [1]:
#%pip install statsforecast pandas polars pyarrow

In [ ]:
import pandas as pd
from utils.preprocessing import load_daily_data, load_monthly_data
from utils.baseline import run_baseline_forecast_daily, run_statistical_forecast_monthly

In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import SimpleExponentialSmoothing, Holt, HoltWinters, AutoRegressive, AutoARIMA

c:\Users\jonas\anaconda3\envs\time_series\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
daily_train, daily_val, daily_test = load_daily_data(use_existing=True)
monthly_train, monthly_val, monthly_test = load_monthly_data(use_existing=True)

### Forecasts:

##### Baseline Forecast:

In [8]:
base_daily_val, base_daily_test = run_baseline_forecast_daily(daily_train, daily_val, daily_test, use_existing=True)
base_monthly_val, base_montly_test = run_statistical_forecast_monthly(monthly_train, monthly_val, monthly_test, use_existing=True)

In [9]:
base_monthly_val

,unique_id,ds,SeasonalNaive,RWD,HistoricAverage,Structural,y
0,Austria,2025-01-01,80.597097,128.681597,76.286243,104.639347,134.178710
1,Austria,2025-02-01,65.414483,129.504484,76.286243,97.459483,140.771071
2,Austria,2025-03-01,62.858710,130.327371,76.286243,96.593040,103.604194
3,Austria,2025-04-01,58.865000,131.150258,76.286243,95.007629,81.323667
4,Austria,2025-05-01,64.170645,131.973144,76.286243,98.071895,71.016774
5,Austria,2025-06-01,68.320000,132.796031,76.286243,100.558016,66.672667
6,Austria,2025-07-01,63.659355,133.618918,76.286243,98.639137,87.980000
7,Austria,2025-08-01,84.477097,134.441805,76.286243,109.459451,74.744194
8,Austria,2025-09-01,82.400667,135.264692,76.286243,108.832679,92.587667
9,Austria,2025-10-01,85.353548,136.087579,76.286243,110.720564,108.226452


##### Statistical Forecast:

In [17]:
sf_daily = StatsForecast(
    models=[
        SimpleExponentialSmoothing(alpha=0.5),
        Holt(season_length=HORIZON_DAILY, error_type="A"),
        HoltWinters(season_length=HORIZON_DAILY, error_type="A"),
        AutoRegressive(lags=HORIZON_DAILY)
    ],
    freq=FREQ_DAILY,
    n_jobs=1
)

stat_daily_val = sf_daily.forecast(df=daily_train, h=HORIZON_DAILY)
stat_daily_test = sf_daily.forecast(df=pd.concat([daily_train, daily_val]), h=HORIZON_DAILY)

stat_daily_val = stat_daily_val.merge(daily_val, on=["unique_id", "ds"], how="left")

stat_daily_val.to_csv("../results/forecasts/stat_daily_val.csv")
stat_daily_test.to_csv("../results/forecasts/stat_daily_test.csv")

: 

: 

In [ ]:
sf_monthly = StatsForecast(
    models=[
        SimpleExponentialSmoothing(alpha=0.5),
        Holt(season_length=HORIZON_MONTHLY, error_type="A"),
        HoltWinters(season_length=HORIZON_MONTHLY, error_type="A"),
        AutoRegressive(lags=HORIZON_MONTHLY)
    ],
    freq=FREQ_MONTHLY,
    n_jobs=1
)

stat_monthly_val = sf_monthly.forecast(df=monthly_train, h=HORIZON_MONTHLY)
stat_monthly_test = sf_monthly.forecast(df=pd.concat([monthly_train, monthly_val]), h=HORIZON_MONTHLY)

stat_monthly_val = stat_monthly_val.merge(monthly_val, on=["unique_id", "ds"], how="left")

stat_monthly_val.to_csv("../results/forecasts/stat_monthly_val.csv")
stat_monthly_test.to_csv("../results/forecasts/stat_monthly_test.csv")